In [1]:
import pandas as pd
df = pd.read_csv('freshmart_products_csv.csv')

In [2]:
df.isnull().sum()

ProductID        0
ProductName      0
Category         0
Price            5
StockQuantity    5
dtype: int64

In [3]:
df = df.fillna('NA')
df.isnull().sum()

ProductID        0
ProductName      0
Category         0
Price            0
StockQuantity    0
dtype: int64

In [4]:
df.iloc[0:5]

,ProductID,ProductName,Category,Price,StockQuantity
0,1,Milk 1,Dairy,19.35,358.0
1,2,Cola 2,Beverages,9.99,81.0
2,3,Banana 3,Produce,30.33,112.0
3,4,Cream 4,Dairy,22.85,224.0
4,5,Cream 5,Dairy,3.85,97.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ProductID      205 non-null    int64 
 1   ProductName    205 non-null    object
 2   Category       205 non-null    object
 3   Price          205 non-null    object
 4   StockQuantity  205 non-null    object
dtypes: int64(1), object(4)
memory usage: 8.1+ KB


In [6]:
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
df['StockQuantity'] = pd.to_numeric(df['StockQuantity'], errors='coerce')
df['StockValue'] = df['Price'] * df['StockQuantity'] #Create new column 'StockValue'

In [7]:
df

,ProductID,ProductName,Category,Price,StockQuantity,StockValue
0,1,Milk 1,Dairy,19.35,358.0,6927.30
1,2,Cola 2,Beverages,9.99,81.0,809.19
2,3,Banana 3,Produce,30.33,112.0,3396.96
3,4,Cream 4,Dairy,22.85,224.0,5118.40
4,5,Cream 5,Dairy,3.85,97.0,373.45
...,...,...,...,...,...,...
200,96,Apple 96,Produce,3.52,494.0,1738.88
201,16,Coffee 16,Beverages,18.95,199.0,3771.05
202,31,Apple 31,Produce,21.83,63.0,1375.29
203,159,Tea 159,Beverages,19.14,242.0,4631.88


In [8]:
df['Price'].mean() #Average price

24.504299999999997

In [9]:
df['StockQuantity'].sum() #Total stock quantity

52810.0

In [10]:
# Database operations and connection
%pip install psycopg2-binary
import psycopg2

def create_connection(
        dbname='FreshMart',
        dbuser='postgres',
        dbpassword='1234',
        dbhost='localhost',
        dbport='5432'
    ):
    conn = psycopg2.connect(
        dbname=dbname,
        user=dbuser,
        password=dbpassword,
        host=dbhost,
        port=dbport
    )
    cursor = conn.cursor()
    print("Connection to database successful")
    return conn, cursor

def create_table(conn, cursor):
    create_table_query = '''
    CREATE TABLE IF NOT EXISTS products (
        id SERIAL PRIMARY KEY,
        product_name VARCHAR(255),
        category VARCHAR(100),
        price DECIMAL,
        stock_quantity INTEGER,
        stock_value NUMERIC
    );
    '''
    cursor.execute(create_table_query)
    conn.commit()
    print("Table created successfully")

def insert_data(conn, cursor, df):
    # Ensure the dataframe columns match the database columns
    for idx, row in df.iterrows():
        cursor.execute('''
            INSERT INTO products (product_name, category, price, stock_quantity, stock_value)
            VALUES (%s, %s, %s, %s, %s)
        ''', (
            row['ProductName'],
            row['Category'],
            row['Price'],
            row['StockQuantity'],
            row['StockValue']
        ))
    conn.commit()
    print("Data inserted successfully")

def fetch_products(conn, cursor):
    cursor.execute("SELECT * FROM products;")
    rows = cursor.fetchall()
    for row in rows:
        print(row)

def close_connection(conn, cursor):
    cursor.close()
    conn.close()
    print("Connection closed")
    

Note: you may need to restart the kernel to use updated packages.


In [16]:
# Ensure StockQuantity is integer and within PostgreSQL INTEGER range
df['StockQuantity'] = df['StockQuantity'].fillna(0).astype(int)
df['StockQuantity'] = df['StockQuantity'].clip(lower=-2147483648, upper=2147483647)
print(df[df['Category'] == 'Dairy'])



     ProductID ProductName Category  Price  StockQuantity  StockValue
0            1      Milk 1    Dairy  19.35            358     6927.30
3            4     Cream 4    Dairy  22.85            224     5118.40
4            5     Cream 5    Dairy   3.85             97      373.45
6            7      Milk 7    Dairy  35.70              0         NaN
9           10    Cream 10    Dairy   1.04            453      471.12
18          19   Butter 19    Dairy  49.18            140     6885.20
19          20   Yogurt 20    Dairy  43.14            144     6212.16
23          24   Butter 24    Dairy  19.89            355     7060.95
24          25    Cream 25    Dairy   5.79            349     2020.71
28          29   Cheese 29    Dairy  20.16             59     1189.44
57          58     Milk 58    Dairy  24.14            110     2655.40
60          61   Butter 61    Dairy  12.56            407     5111.92
64          65     Milk 65    Dairy  10.88            361     3927.68
69          70   Yog

In [ ]:
# Ensure StockQuantity is integer and within PostgreSQL INTEGER range
df['StockQuantity'] = df['StockQuantity'].fillna(0).astype(int)
df['StockQuantity'] = df['StockQuantity'].clip(lower=-2147483648, upper=2147483647)
print(df[df['StockQuantity'] < 50])

     ProductID        ProductName   Category  Price  StockQuantity  StockValue
6            7             Milk 7      Dairy  35.70              0         NaN
29          30    Orange Juice 30  Beverages  33.46             11      368.06
56          57           Bread 57     Bakery  14.46             44      636.24
69          70          Yogurt 70      Dairy  38.02             45     1710.90
88          89     Granola Bar 89     Snacks  28.35              0         NaN
114        115           Milk 115      Dairy  20.48             48      983.04
120        121      Croissant 121     Bakery   9.57              0         NaN
125        126          Chips 126     Snacks  32.02             18      576.36
129        130        Lettuce 130    Produce  39.21              0         NaN
145        146          Apple 146    Produce  32.18             31      997.58
146        147          Chips 147     Snacks  12.96             47      609.12
153        154    Granola Bar 154     Snacks  25.46 